# NOTEBOOK porównywanie dwóch ze sobą
- porównać wizualnie
- porównać wartości
- miary statystyczne

### Input json format

```
data = {
            "n_photons": Nphotons,
            "overflow": cube_overflow,
            "bins_per_1_cm": bins_per_1_cm,
            "cube": Cube
        }
```

In [1]:
import json

In [2]:
filename_mati_cube = "mati_cube.json"
filename_t1_cube = "mc456_mc_1mln_my_params_cube.json"

file_mati_cube = open(filename_mati_cube, 'r')
file_t1_cube = open(filename_t1_cube, 'r')

mati_cube = json.load(file_mati_cube)
t1_cube = json.load(file_t1_cube)

file_mati_cube.close()
file_t1_cube.close()